<a href="https://colab.research.google.com/github/Azlan-Qaisrani/Langgraph/blob/main/AGENTS.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install langgraph langchain-core langgraph-prebuilt sentence-transformers faiss-cpu python-dotenv

In [2]:
!pip install langchain-google-genai

In [3]:
from typing_extensions import TypedDict
from langgraph.graph.message import add_messages
from typing import List,TypedDict
from langchain_core.messages import HumanMessage
from langgraph.graph import StateGraph, START, END
from langchain_google_genai import ChatGoogleGenerativeAI

In [4]:

import os
os.environ["GRPC_VERBOSITY"] = "NONE"
os.environ["GRPC_TRACE"] = ""
os.environ["GOOGLE_CLOUD_PROJECT"] = ""          # ← This kills the 404 error
os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = ""  # extra safety
from google.colab import userdata


llm = ChatGoogleGenerativeAI(
    model="gemini-2.0-flash",
    google_api_key=userdata.get("google_api_key"),
    temperature=0.7
)

In [5]:
class AgentState(TypedDict):
    messages: List[HumanMessage]

In [6]:
def process(state: AgentState) -> AgentState:
    response = llm.invoke(state["messages"])
    print(f"\nAI: {response.content}")
    return state

In [7]:
graph = StateGraph(AgentState)
graph.add_node("process", process)
graph.add_edge(START, "process")
graph.add_edge("process", END)
agent = graph.compile()

In [8]:
user_input = input("Enter: ")
while user_input != "exit":
    agent.invoke({"messages": [HumanMessage(content=user_input)]})
    user_input = input("Enter: ")

Enter: hi

AI: Hi there! How can I help you today?
Enter: my name is  Azlan tell me a joke

AI: Okay, Azlan, here's a joke for you:

Why don't scientists trust atoms?

Because they make up everything!
Enter: now can you explain langgraph

AI: Okay, let's break down LangGraph. In essence, LangGraph is a framework designed to make it easier to build and manage complex, multi-agent conversations and workflows using Large Language Models (LLMs). It's like a graph-based orchestrator for LLM-powered applications.

Here's a more detailed explanation, covering the key concepts:

**1. The Problem LangGraph Solves:**

*   **Complex Conversations:** Many LLM applications go beyond simple question-and-answer interactions. They involve multi-step processes, loops, decision-making based on previous outputs, and potentially multiple agents interacting with each other.
*   **Orchestration Challenges:**  Managing this complexity with traditional programming approaches can be cumbersome and difficult to

In [9]:
from langchain_core.messages import AIMessage
from typing import Union
class AgentState(TypedDict):
  message:List[Union[HumanMessage,AIMessage]]



In [10]:
def process(state: AgentState) -> AgentState:
    response = llm.invoke(state["message"])
    state['message'].append(AIMessage(content=response.content))

    print(f"\nAI: {response.content}")
    print("CURRENTSTATE: " ,state['message'])
    return state


In [11]:
graph = StateGraph(AgentState)
graph.add_node("process", process)
graph.add_edge(START, "process")
graph.add_edge("process", END)
agent = graph.compile()

In [12]:
conversation_history=[]
user_input=input("Enter: ")
while user_input != "exit":
    conversation_history.append(HumanMessage(content=user_input))
    results=agent.invoke({"message": conversation_history})
    conversation_history=results['message']
    user_input = input("Enter: ")

Enter: hi

AI: Hi there! How can I help you today?
CURRENTSTATE:  [HumanMessage(content='hi', additional_kwargs={}, response_metadata={}), AIMessage(content='Hi there! How can I help you today?', additional_kwargs={}, response_metadata={})]
Enter: myself AZLAN QAISRANI

AI: It's nice to meet you, Azlan Qaisrani! How can I help you today? Is there anything you'd like to know, or anything I can do for you?
CURRENTSTATE:  [HumanMessage(content='hi', additional_kwargs={}, response_metadata={}), AIMessage(content='Hi there! How can I help you today?', additional_kwargs={}, response_metadata={}), HumanMessage(content='myself AZLAN QAISRANI', additional_kwargs={}, response_metadata={}), AIMessage(content="It's nice to meet you, Azlan Qaisrani! How can I help you today? Is there anything you'd like to know, or anything I can do for you?", additional_kwargs={}, response_metadata={})]
Enter: define MACHine learning

AI: Machine learning (ML) is a subfield of artificial intelligence (AI) that foc

In [13]:
with open("logging.txt", "w") as file:
    file.write("Your Conversation Log:\n")

    for message in conversation_history:
        if isinstance(message, HumanMessage):
            file.write(f"You: {message.content}\n")
        elif isinstance(message, AIMessage):
            file.write(f"AI: {message.content}\n\n")
    file.write("End of Conversation")

print("Conversation saved to logging.txt")

Conversation saved to logging.txt
